In [60]:
import pandas as pd
import numpy as np

data = pd.read_csv('online_retail_II.csv')

print(data.head())
print(data.info())

  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

           InvoiceDate  Price  Customer ID         Country  
0  2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1  2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2  2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
3  2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
4  2009-12-01 07:45:00   1.25      13085.0  United Kingdom  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice 

In [61]:
data.dropna(subset=['Customer ID', 'InvoiceDate', 'Quantity', 'Price'], inplace=True)

data['TotalPrice'] = data['Quantity'] * data['Price']

median_spent = data.groupby('Customer ID')['TotalPrice'].sum().median()
data['PurchaseClass'] = data.groupby('Customer ID')['TotalPrice'].transform('sum').apply(lambda x: 1 if x > median_spent else 0)

In [62]:
X = data[['Quantity', 'Price']].values  
y = data['PurchaseClass'].values 

train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [63]:
class NaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_probs = {}
        self.feature_probs = {}

        for c in self.classes:
            X_c = X[y == c]
            self.class_probs[c] = len(X_c) / len(y)

            feature_probs_c = []
            for j in range(X.shape[1]):
                mean = np.mean(X_c[:, j])
                var = np.var(X_c[:, j])
                feature_probs_c.append((mean, var))
            self.feature_probs[c] = feature_probs_c

    def predict(self, X):
        y_pred = []
        for x in X:
            probs = []
            for c in self.classes:
                prob = np.log(self.class_probs[c])
                for j in range(len(x)):
                    mean, var = self.feature_probs[c][j]
                    prob += np.log(self.gaussian_likelihood(x[j], mean, var))
                probs.append(prob)
            y_pred.append(self.classes[np.argmax(probs)])
        return np.array(y_pred)

    def gaussian_likelihood(self, x, mean, var):
        return (1 / np.sqrt(2 * np.pi * var)) * np.exp(-(x - mean) ** 2 / (2 * var))

In [66]:
nb = NaiveBayes()
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy:.2f}")

C:\Users\rveab\AppData\Local\Temp\ipykernel_21452\2478300304.py:30: RuntimeWarning: divide by zero encountered in log
  prob += np.log(self.gaussian_likelihood(x[j], mean, var))


Accuracy: 0.85
